# Entrenamiento de Tenis para elegir pista 

## Conjunto de datos

### Descripción del conjunto de datos y tarea a realizar
Para este proyecto vamos a utilizar el conjunto de datos de los partidos realizados en la ATP desde el año 2010 hasta el 2021, con esto realizaremos una serie de modelos de clasificación con los que predecir en base a las caracteristicas de un jugador vencedor y de otro perdedor, además del tiempo de duración del partido y el ranking de la ATP, cual sería el tipo de superficie más idonea para que el resultado planteado ocurriera.

In [2]:
from pyspark.sql.functions import *
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from IPython.core.display import display, HTML

display(HTML("<style>.container { width:100% !important; }</style>"))
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

### Cargamos el conjunto de datos con todos los partidos de la ATP desde 2010 hasta 2021
all_atp_games = spark.read.csv("file:///home/alumno/Desktop/pyspark_mllib_regresion_project/datasets/all_atp_games.csv", header=True, inferSchema=True, sep=",")

### Ahora realizamos una limpieza de las columnas que no vamos a utilizar
# primero eliminamos las columnas que nos dan información sobre el torneo o datos externos al juego en si
columns_to_drop = ['tourney_name', 'tourney_id', 'draw_size', 'match_num', 'winner_id', 'loser_id', 'winner_name', 'loser_name',
                  'winner_seed', 'winner_entry', 'winner_ioc', 'loser_seed', 'loser_entry', 'loser_ioc',
                  'best_of', 'round']
all_atp_games = all_atp_games.drop(*columns_to_drop)

# vamos a eliminar todos los registros los cuales no cuentes con la altura del vencedor o perdedor
all_atp_games = all_atp_games.dropna('any', subset=['winner_ht', 'loser_ht'])
# tambien los datos sin el terreno definido los eliminaremos
all_atp_games = all_atp_games.dropna('all', subset=['surface'])

# creamos una nueva columna con el año que ocurrió el partido
all_atp_games = all_atp_games.withColumn("year", expr('LEFT(tourney_date,4)'))


all_atp_games.limit(5)

surface,tourney_level,tourney_date,winner_hand,winner_ht,winner_age,loser_hand,loser_ht,loser_age,score,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,year
Hard,A,20100103,R,188.0,27.3483915127,R,183.0,30.3463381246,7-6(5) 6-2,84.0,15.0,0.0,63.0,42.0,36.0,14.0,10.0,3.0,3.0,4.0,2.0,56.0,34.0,29.0,11.0,10.0,3.0,5.0,7.0,4410.0,77.0,598.0,2010
Hard,A,20100103,L,198.0,22.5434633812,L,190.0,22.3709787817,7-5 6-1,70.0,10.0,3.0,57.0,30.0,23.0,19.0,10.0,0.0,0.0,2.0,2.0,66.0,34.0,22.0,14.0,9.0,7.0,10.0,134.0,400.0,78.0,590.0,2010
Hard,A,20100103,R,185.0,23.5482546201,L,185.0,28.4517453799,6-3 4-6 6-4,121.0,5.0,4.0,97.0,51.0,33.0,27.0,15.0,5.0,8.0,4.0,0.0,85.0,58.0,38.0,14.0,14.0,7.0,11.0,52.0,850.0,88.0,568.0,2010
Hard,A,20100103,R,188.0,22.1081451061,L,183.0,28.6214921287,7-5 6-1,64.0,12.0,1.0,50.0,35.0,30.0,12.0,10.0,3.0,3.0,2.0,1.0,53.0,29.0,16.0,15.0,9.0,2.0,5.0,285.0,151.0,28.0,1260.0,2010
Hard,A,20100103,R,196.0,24.2984257358,R,183.0,21.4291581109,6-2 6-4,69.0,3.0,1.0,46.0,27.0,24.0,14.0,9.0,1.0,1.0,6.0,1.0,69.0,41.0,26.0,14.0,9.0,6.0,9.0,20.0,1655.0,251.0,179.0,2010


### Operaciones con API de datos estructurados

In [ ]:
# Para las variables de respuesta (categorica), calcular estadísticas agregadas de las variables predictoras
# en base a la de respuesta

# Vamos a observar la media, minimo y máximo de edad y altura del ganador y perdedor en cada tipo de superficie
height_by_surface = all_atp_games.groupBy(col('surface')).agg(expr('mean(winner_ht)'),expr('mean(loser_ht)'),
                                         expr('max(winner_ht)'),expr('max(loser_ht)'),
                                         expr('min(winner_ht)'),expr('min(loser_ht)'))

age_by_surface = all_atp_games.groupBy(col('surface')).agg(expr('mean(winner_age)'),expr('mean(loser_age)'),
                                         expr('max(winner_age)'),expr('max(loser_age)'),
                                         expr('min(winner_age)'),expr('min(loser_age)'))

rank_by_surface = all_atp_games.groupBy(col('surface')).agg(expr('mean(winner_rank)'),expr('mean(loser_rank)'),
                                         expr('max(winner_rank)'),expr('max(loser_rank)'),
                                         expr('min(winner_rank)'),expr('min(loser_rank)'))


height_by_surface
age_by_surface.show()
rank_by_surface.show()

Se puede observar independientemente del tipo de pista, los ganadores son de media más altos, más jovenes y con una mejor posición en el ranking.
A excepción de los jugadores en Grass (Hierba) donde la media de los jugadores ganadores son mayores, esto es interesante aunque habría que analizar con más profundidad para deducir algo.

In [ ]:
# vamos a limpiar todos los valores null ya que no queremos ni valores nulos y 0 introducidos en nulos para el entrenamiento
all_atp_games = all_atp_games.dropna('any')
# Contamos con demasiados registros, así que vamos a reducir esto, primero vamos a ver cuantos registros tenemos de cada año
print("Datos totales:", all_atp_games.count())
all_atp_games.groupBy("year").count().show()
# contamos con 12 años diferentes, vamos a tratar de hacer un muestreo estratificado para obtener
# un conjunto reducido.
porcentaje_por_clase = 0.125
seed_stratified = 12
fracciones = all_atp_games.select("year").distinct().withColumn("fraccion", lit(porcentaje_por_clase)).rdd.collectAsMap()
all_atp_games_reducido = all_atp_games.stat.sampleBy("year", fracciones, seed_stratified)

# vamos a comprobar la distribución y total de muestras actuales
print("Nuevos datos reducidos:", all_atp_games_reducido.count())
all_atp_games_reducido.groupBy("year").count().show()

In [ ]:
# Vamos a proyectar con las variables de winner_rank, winner_ht, loser_rank, loser_ht y minutes. 
# Ademas de surface (variable a predecir).
projected_games = all_atp_games_reducido.select('surface','winner_ht','loser_ht','minutes','winner_rank','loser_rank')
projected_games.show()

### Analisis variable a variable

In [ ]:
# altura
height_resume_winner = all_atp_games.selectExpr('mean(winner_ht) as mean_winner', 'percentile(winner_ht, 0.5) as median_winner',
                                         'percentile(winner_ht, 0.25) as Q1_winner', 'percentile(winner_ht, 0.75) as Q3_winner',
                                        'kurtosis(winner_ht) as kurtosis_winner', 'skewness(winner_ht) as skewness_winner')

height_resume_loser = all_atp_games.selectExpr('mean(loser_ht) as mean_loser', 'percentile(loser_ht, 0.5) as median_loser',
                                         'percentile(loser_ht, 0.25) as Q1_loser', 'percentile(loser_ht, 0.75) as Q3_loser',
                                        'kurtosis(loser_ht) as kurtosis_loser', 'skewness(loser_ht) as skewness_loser')
# edad
age_resume_winner = all_atp_games.selectExpr('mean(winner_age) as mean_winner', 'percentile(winner_age, 0.5) as median_winner',
                                         'percentile(winner_age, 0.25) as Q1_winner', 'percentile(winner_age, 0.75) as Q3_winner',
                                        'kurtosis(winner_age) as kurtosis_winner', 'skewness(winner_age) as skewness_winner')

age_resume_loser = all_atp_games.selectExpr('mean(loser_age) as mean_loser', 'percentile(loser_age, 0.5) as median_loser',
                                         'percentile(loser_age, 0.25) as Q1_loser', 'percentile(loser_age, 0.75) as Q3_loser',
                                        'kurtosis(loser_age) as kurtosis_loser', 'skewness(loser_age) as skewness_loser')

# rank
rank_resume_winner = all_atp_games.selectExpr('mean(winner_rank) as mean_winner', 'percentile(winner_rank, 0.5) as median_winner',
                                         'percentile(winner_rank, 0.25) as Q1_winner', 'percentile(winner_rank, 0.75) as Q3_winner',
                                        'kurtosis(winner_rank) as kurtosis_winner', 'skewness(winner_rank) as skewness_winner')

rank_resume_loser = all_atp_games.selectExpr('mean(loser_rank) as mean_loser', 'percentile(loser_rank, 0.5) as median_loser',
                                         'percentile(loser_rank, 0.25) as Q1_loser', 'percentile(loser_rank, 0.75) as Q3_loser',
                                        'kurtosis(loser_rank) as kurtosis_loser', 'skewness(loser_rank) as skewness_loser')

# minute
minute_resume = all_atp_games.selectExpr('mean(minutes) as mean', 'percentile(minutes, 0.5) as median',
                                         'percentile(minutes, 0.25) as Q1', 'percentile(minutes, 0.75) as Q3',
                                        'kurtosis(minutes) as kurtosis', 'skewness(minutes) as skewness')
minute_resume_by_surface = all_atp_games.groupBy('surface').agg(expr('mean(minutes) as mean'), expr('percentile(minutes, 0.5) as median'),
                                                               expr('percentile(minutes, 0.25) as Q1'),
                                                               expr('percentile(minutes, 0.75) as Q3'),
                                                               expr('kurtosis(minutes) as kurtosis'),
                                                               expr('skewness(minutes) as skewness'),
                                                               expr('count(minutes) as count'))

In [ ]:
# vamos a transformar el conjunto de datos a un df de pandas para observar el histograma de cada variable
all_atp_games_df = all_atp_games.toPandas()

print("resumen height")
height_resume_winner.show()
all_atp_games_df.hist(column = "winner_ht")
height_resume_loser.show()
all_atp_games_df.hist(column = "loser_ht")

La altura de los jugadores parece seguir una distribución aproximadamente normal observando su histograma, kurtosis y asimetría.

In [ ]:
print("resumen age")
age_resume_winner.show()
all_atp_games_df.hist(column = "winner_age")
age_resume_loser.show()
all_atp_games_df.hist(column = "loser_age")

La edad de los jugadores parece seguir una distribución aproximadamente normal observando su histograma, kurtosis y asimetría.

In [ ]:
print("resumen rank")
rank_resume_winner.show()
all_atp_games_df.hist(column = "winner_rank")
rank_resume_loser.show()
all_atp_games_df.hist(column = "loser_rank")

El ranking de los jugadores no es normal, pero esto es normal ya que estamos trabajando con todas las partidas, y dado que el rango que necesitan los jugadores para participar en estos tipos de torneos es alto, obviamente en la mayor parte de los partidos están todos agrupados en los primeros 200 del ATP.

In [ ]:
print("resumen game lenght")
minute_resume.show()
all_atp_games_df.hist(column = "minutes")

Observando la duración de las partidas, podemos observar que suelen durar una hora y 40 minutos de media, 

In [ ]:
print("resumen game lenght by surface")
minute_resume_by_surface.show()

Al observar la duración de las partidas por el tipo de pista, podemos ver que la pista de alfombra tiene una duración más alta de media, esto podemos ver que se debe a que únicamente tenemos 1 registro de este tipo de pista.
Por otro lado se observa que en cesped la media es ligeramente superior, lo cual puede deberse a que cesped es el tipo de cancha más rápido, lo que pueda ayudar a que los jugadores puedan aguantar más en cada punto y esto haga que dure más la partida.

In [ ]:
hand_resume_winner_surface = all_atp_games.groupBy('winner_hand').agg(count('surface').alias('count'), (count('surface')/all_atp_games.count()).alias('percentage'))
hand_resume_loser_surface = all_atp_games.groupBy('loser_hand').agg(count('surface').alias('count'), (count('surface')/all_atp_games.count()).alias('percentage'))

hand_resume_winner_surface.show()
hand_resume_loser_surface.show()


all_atp_games.crosstab('tourney_level','winner_hand').orderBy('tourney_level_winner_hand', ascending=True).show(n=50)

all_atp_games.filter("winner_rank < 50").crosstab('winner_rank','winner_hand').orderBy('winner_rank_winner_hand', ascending=True).show(n=20)


Podemos observar que la dominancia de la mano derecha es constante en todos los niveles de juego y todos los jugadores que han tenido cierto rango en el ranking ATP, lo que es entendible ya que generalmente la gente suele ser diestra.

## Entrenamiento y evaluación de modelo

En la tarea de clasificación, teniendo en cuenta las siguientes variables de los jugadores (a la hora de introducir los datos habrá que predefinir dos jugadores, uno como ganador y otro como perdedor):
- hand: mano dominante
- tourney_level: Nivel de la competición donde se juega el partido
- ht: altura
- age: edad
- rank: posición en el ranking ATP
- minutes: duración en minutos de la partida

Trataremos de predecir que tipo de superficie sería la más idonea para que el jugador introducido como vencedor obtenga la victoria en la duración de partida introducida:
- surface: tipo de pista


In [ ]:
import pyspark.ml.feature as ft 
from pyspark.ml import Pipeline

# preparamos el dataset con las variables que vamos a utilizar
games_df = all_atp_games_reducido.select('winner_hand','loser_hand','winner_ht','loser_ht','winner_age','loser_age','winner_rank','loser_rank','minutes','tourney_level','surface')
games_df = games_df.withColumnRenamed("winner_hand","w_hand")
games_df = games_df.withColumnRenamed("loser_hand","l_hand")
games_df = games_df.withColumnRenamed("winner_ht","w_height")
games_df = games_df.withColumnRenamed("loser_ht","l_height")
games_df = games_df.withColumnRenamed("winner_age","w_age")
games_df = games_df.withColumnRenamed("loser_age","l_age")
games_df = games_df.withColumnRenamed("winner_rank","w_rank")
games_df = games_df.withColumnRenamed("loser_rank","l_rank")
games_df.show(5)
games_df.summary().show()

In [ ]:
# numero total de clases a predecir
clases = games_df.groupBy('surface').agg(expr('count(minutes) as count'))
clases.show()

In [ ]:
# ahora cargamos un conjunto de entrenamiento y otro de test
train_df, test_df = games_df.randomSplit(weights = [0.70, 0.30], seed = 2021)
print("conjunto de entrenamiento: ", train_df.count(), ", conjunto de test: ", test_df.count())

### ESTIMADORES Y TRANSFORMADORES

In [ ]:
from pyspark.ml.linalg import Vectors
### ESTIMADORES Y TRANSFORMADORES
# binarizer
binarizer_rank = ft.Binarizer(threshold=50,inputCols=["w_rank","l_rank"], outputCols=["w_rank_bin","l_rank_bin"])
binarizer_min = ft.Binarizer(threshold=108,inputCol="minutes", outputCol="minutes_bin")
# string indexer
indexer = ft.StringIndexer(inputCols=['w_hand','l_hand','tourney_level','surface'],outputCols=['w_hand_class','l_hand_class','tourney_level_class','surface_class'])
# one hot encoder
onehotencoder = ft.OneHotEncoder(inputCols=['w_hand_class','l_hand_class','tourney_level_class'],outputCols=['w_hand_vec','l_hand_vec','tourney_level_vec'])
# assembler
assembler = ft.VectorAssembler(inputCols=["w_rank","l_rank","w_rank_bin","l_rank_bin",
                                          "minutes","minutes_bin",
                                          "w_height","l_height",
                                          "w_age","l_age",
                                          "w_hand_vec","l_hand_vec",
                                         "tourney_level_vec"], outputCol="features")
# Principal Component Analysis
######## Para elegir cuantos componentes elegir, he realizado el entrenamiento hasta pca, analizado de la siguiente manera variable a variable
######## pca_model = pca.fit(test_predictions_df)
######## pca_model.explainedVariance
######## >> DenseVector([0.4912, 0.2797, 0.2166, 0.0052, 0.0047, 0.0014, 0.0011, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
######## Aquí podemos obtener que pocentaje de conjunto es explicado por cada variable
######## y he concluido que elegiría los 7 primeros componentes
pca = ft.PCA(k=7, inputCol="features", outputCol="pcaFeatures")
# Standard Scaler
scaler = ft.StandardScaler(withMean=True, withStd=True, inputCol="pcaFeatures",outputCol="std_features")

# Pipeline of estimators and transformers
pipeline = Pipeline(stages=[binarizer_rank, binarizer_min, indexer, onehotencoder, assembler, pca, scaler])
fitted_model = pipeline.fit(train_df)
test_predictions_df = fitted_model.transform(train_df)

test_predictions_df = test_predictions_df.drop('w_hand','l_hand',
                                               'tourney_level',
                                               'w_hand_class','l_hand_class',
                                               'tourney_level_class',
                                               'w_height','l_height',
                                               'w_age','l_age',
                                               'w_rank','l_rank',
                                               'minutes','minutes_bin',
                                               'w_rank_bin','l_rank_bin',
                                               'w_hand_vec','l_hand_vec',
                                               'tourney_level_vec',
                                               'features',
                                               'pcaFeatures')

test_predictions_df.show(5, truncate=False)



#random_forest = RandomForestClassifier(featuresCol="std_features",labelCol="class", predictionCol="prediction")

### Modelo de Regresión Logística (Multinomial)

La regresión logística multinomial generaliza el metodo de regresión logística para problemas con multiples clases, es decir, se utiliza para predecir las probabilidades de los diferentes resultados posibles de una distribución categórica como variable dependiente, dado un conjunto de variables independientes.

### Modelo de Random Forest

Esto hacen los arboles aleatorios

In [ ]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

random_forest = RandomForestClassifier(featuresCol="std_features", labelCol="surface_class",predictionCol="prediction")

# tuberia con el modelo de random forest
rf_pipeline = Pipeline(stages=[binarizer_rank, binarizer_min, indexer, onehotencoder, assembler, pca, scaler, random_forest])

# parametros a probar en la validación cruzada
rf_parameters = (ParamGridBuilder()
                .addGrid(random_forest.maxDepth, [4, 6, 8, 10, 12, 14])
                .addGrid(random_forest.maxBins, [20, 22, 24, 25, 27])
                .addGrid(random_forest.numTrees, [5, 6, 7, 8])
                .build())

# evaluador para probar la precisión del modelo
rf_evaluator = MulticlassClassificationEvaluator(predictionCol="prediction", labelCol="surface_class", metricName="accuracy")

# configuración de la validación cruzada
cross_validator = CrossValidator(estimator = rf_pipeline,
                                estimatorParamMaps = rf_parameters,
                                evaluator = rf_evaluator,
                                numFolds = 5)

# realizar la validación cruzada
rf_model = cross_validator.fit(train_df)
best_random_forest = rf_model.bestModel

In [ ]:
# Vamos a comprobar la precisión con el conjunto de test
results = []
best_results = []
for (result, config) in zip(rf_model.avgMetrics, rf_parameters):
    if result > 0.90:
        best_results.append([result, config[random_forest.maxDepth], config[random_forest.maxBins], config[random_forest.numTrees]])
    results.append([result, config[random_forest.maxDepth], config[random_forest.maxBins], config[random_forest.numTrees]])

sorted(best_results, key=lambda x: x[1])

In [ ]:
# Vamos a observar el modelo con mayor precisión
random_forest_predictions = best_random_forest.transform(test_df)
print("Random Forest: ", rf_evaluator.evaluate(random_forest_predictions))

### Modelo de Gradient-Boosted Tree

Esto hacen el Gradient-Boosted Tree

#### Descripción de modelos seleccionados
- Regresion logística:

- Random Forest

Random Forest es un metodo de aprendizaje para clasificación, regresión y otras tareas que se centran en construir multiples arboles de decisión a la hora de entrenar y sacando la clase que sea la moda de los arboles individuales.


- Gradient-boosted tree